### Предобработка данных

Перед тем как обучать модель, нужно подготовить для этого данные. 

Современные нейонные сети, и в особенности генеративные модели, обучаются на очень больших данных. Обработка их с помощью всем известной библиотеки `pandas` затруднительна по ряду причин (нет многопточности, кеширования, неудобный формат хранения, ...)

Для работы с большими объемами структурированных данных существует несколько библиотек (`arrow`, `pil`, ...). В этом ноутбуке мы будем использовать библиотеку `datasets` от *hugging-face*, основанную на `arrow`. 

In [1]:
from pathlib import Path
from datasets import load_dataset

/Users/d.tsimerman/blowjob/pycon-chit-chat/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset('csv', 
    data_files={
        'train': '../data.csv'
    })

Using custom data configuration default-4b39c19aab5c9a43
Reusing dataset csv (/Users/d.tsimerman/.cache/huggingface/datasets/csv/default-4b39c19aab5c9a43/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 1/1 [00:00<00:00, 35.37it/s]


In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['context_3', 'context_2', 'context_1', 'response'],
        num_rows: 2507
    })
})

In [4]:
# train_test_split
data = data['train'].train_test_split(test_size=0.2)  # YOUR CODE HERE

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['context_3', 'context_2', 'context_1', 'response'],
        num_rows: 2005
    })
    test: Dataset({
        features: ['context_3', 'context_2', 'context_1', 'response'],
        num_rows: 502
    })
})

In [6]:
data['train'][0:2]  # First two rows of the dataset of type Dict[FeatureName, List[values]]

{'context_3': ['Чем запомнился день?', None],
 'context_2': ['Котик живот кусал))', None],
 'context_1': ['Оте мило)',
  'Будет маска. А то прыщ выскочил .\nГоворят помогает.'],
 'response': [None, 'Мне кожу сушит']}

Обработка данных осуществляется с помощью 2х основных функций -- `filter` и `map` . 

In [7]:
? data.filter

Signature:
 data.filter(
    function,
    with_indices=False,
    input_columns: Union[str, List[str], NoneType] = None,
    batched: bool = False,
    batch_size: Union[int, NoneType] = 1000,
    keep_in_memory: bool = False,
    load_from_cache_file: bool = True,
    cache_file_names: Union[Dict[str, Union[str, NoneType]], NoneType] = None,
    writer_batch_size: Union[int, NoneType] = 1000,
    fn_kwargs: Union[dict, NoneType] = None,
    num_proc: Union[int, NoneType] = None,
    desc: Union[str, NoneType] = None,
) -> 'DatasetDict'
Docstring:
Apply a filter function to all the elements in the table in batches
and update the table so that the dataset only includes examples according to the filter function.
The transformation is applied to all the datasets of the dataset dictionary.

Args:
    function (`callable`): with one of the following signature:
        - ``function(example: Dict[str, Any]) -> bool`` if ``with_indices=False, batched=False``
        - ``function(example: Dict

In [8]:
? data.map

Signature:
 data.map(
    function: Union[Callable, NoneType] = None,
    with_indices: bool = False,
    with_rank: bool = False,
    input_columns: Union[str, List[str], NoneType] = None,
    batched: bool = False,
    batch_size: Union[int, NoneType] = 1000,
    drop_last_batch: bool = False,
    remove_columns: Union[str, List[str], NoneType] = None,
    keep_in_memory: bool = False,
    load_from_cache_file: bool = True,
    cache_file_names: Union[Dict[str, Union[str, NoneType]], NoneType] = None,
    writer_batch_size: Union[int, NoneType] = 1000,
    features: Union[datasets.features.features.Features, NoneType] = None,
    disable_nullable: bool = False,
    fn_kwargs: Union[dict, NoneType] = None,
    num_proc: Union[int, NoneType] = None,
    desc: Union[str, NoneType] = None,
) -> 'DatasetDict'
Docstring:
Apply a function to all the elements in the table (individually or in batches)
and update the table (if function does updated examples).
The transformation is applied to a

Оставим только те примеры, где context_1 не пустой

In [9]:
data = data.filter(lambda sample: sample['context_1'] is not None and sample['response'] is not None)

100%|██████████| 1/1 [00:00<00:00, 101.73ba/s]


In [10]:
data['train'][:2]

{'context_3': [None, 'Про другой орех заговорили😂'],
 'context_2': [None, 'Какой???'],
 'context_1': ['Будет маска. А то прыщ выскочил .\nГоворят помогает.',
  'Да про попу😂'],
 'response': ['Мне кожу сушит',
  'Я походу в леди превращаюсь))) скромную и нудную😂😂😂😂']}

Приводим каждый пример из датасета к виду одной строки

In [11]:
from typing import Dict

FIRST_SPEAKER_TOKEN = '@@ПЕРВЫЙ@@'
SECOND_SPEAKER_TOKEN = '@@ВТОРОЙ@@'

CONTEXT_COLS = ['context_3', 'context_2', 'context_1']
RESPONSE_COL = ['response']

def convert_to_dialog(sample: Dict[str, str]) -> Dict[str, str]:
    """
        Convert sample row to dialogs str format
    """
    text = FIRST_SPEAKER_TOKEN + ' '
    for col in CONTEXT_COLS + RESPONSE_COL:
        next_special_token = FIRST_SPEAKER_TOKEN if text.strip().endswith(SECOND_SPEAKER_TOKEN) else SECOND_SPEAKER_TOKEN

        if (col not in sample) or (not sample[col]):
            continue

        text += sample[col]
        if col != RESPONSE_COL[0]:
            text += ' ' + next_special_token + ' '
    
    return {'text': text}

assert convert_to_dialog(
    {
        'context_3': 'привет',
        'context_2': 'привет!',
        'context_1': 'как дела?',
        'response': 'супер)'
    }
) == {'text': '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет! @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}
assert convert_to_dialog(
    {
        'context_1': 'как дела?',
        'response': 'супер)'
    }
) == {'text': '@@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}

In [12]:
data = data.map(convert_to_dialog, remove_columns=CONTEXT_COLS + RESPONSE_COL)

100%|██████████| 325/325 [00:00<00:00, 7789.51ex/s]


In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1334
    })
    test: Dataset({
        features: ['text'],
        num_rows: 325
    })
})

Теперь датасет необходимо токенизировать, точно также как мы это делали при знакомстве с хаггинфейс

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from typing import List


def tokenize_sample(sample: Dict[str, str]):
    return tokenizer(sample['text'], max_length=512, truncation=True, padding=True)

In [16]:
data = data.map(tokenize_sample, remove_columns=['text'])

100%|██████████| 325/325 [00:00<00:00, 3674.44ex/s]


In [17]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1334
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 325
    })
})

### Обучение

Создаем модель, которую будем обучать, а также вспомогательные классы -- trainer, training_args и datacollator

In [18]:
import torch
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-small').to(device)

Указываем основные параметры обучения и создаем класс Trainer (https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer)

In [19]:
arguments = {
    'output_dir': './training_output',  # path to save the model's checkpoints
    'per_device_train_batch_size': 4,  # batch size per GPU/CPU for training
    'gradient_accumulation_steps': 4,  # number of batches to accumulate gradient
    'max_steps': 10,  # total number of optimizer.step() calls
    'save_steps': 10,  # save every save_steps
    'eval_steps': 10,  # run evaluation every eval_steps
    'dataloader_num_workers': 0,  # number of workers for data loading (default: 0)
    'save_total_limit': 2,  # total number of checkpoints to save, delete older checkpoints when reached
}

trainer = Trainer(
    model=model,
    args=TrainingArguments(**arguments),
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=data['train'],
    eval_dataset=data['test']
)

max_steps is given, it will override any value given in num_train_epochs


Запускаем обучение

In [20]:
trainer.train()

/Users/d.tsimerman/blowjob/pycon-chit-chat/.venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1334
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 10
100%|██████████| 10/10 [00:34<00:00,  3.29s/it]Saving model checkpoint to ./training_output/checkpoint-10
Configuration saved in ./training_output/checkpoint-10/config.json
Model weights saved in ./training_output/checkpoint-10/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 10/10 [00:35<00:00,  3.58s/it]

{'train_runtime': 35.7758, 'train_samples_per_second': 4.472, 'train_steps_per_second': 0.28, 'train_loss': 4.498283386230469, 'epoch': 0.12}


TrainOutput(global_step=10, training_loss=4.498283386230469, metrics={'train_runtime': 35.7758, 'train_samples_per_second': 4.472, 'train_steps_per_second': 0.28, 'train_loss': 4.498283386230469, 'epoch': 0.12})

#### Пробуем обученный чекпоинт

In [21]:
from transformers import AutoModelWithLMHead

checkpoint_path = './training_output/checkpoint-10/'

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small')
model = AutoModelWithLMHead.from_pretrained(checkpoint_path)

loading file https://huggingface.co/tinkoff-ai/ruDialoGPT-small/resolve/main/vocab.json from cache at /Users/d.tsimerman/.cache/huggingface/transformers/7b39d42ced5110cdcc7e4bdb46c98242413a9f53c1f19df10b3445e53e563090.c483bc3440d25937fdac74506b73b76ee6e67f778a804756214363fc2a1a66ef
loading file https://huggingface.co/tinkoff-ai/ruDialoGPT-small/resolve/main/merges.txt from cache at /Users/d.tsimerman/.cache/huggingface/transformers/d0bf521c0f60cb2c5e95da8868245bd528cd0c1cf829616231b00e0b5bdca3cd.7362c0dbb32f750eeea5a5b93bbd0c6876eac41453369265d5a49df1c9142b6f
loading file https://huggingface.co/tinkoff-ai/ruDialoGPT-small/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/tinkoff-ai/ruDialoGPT-small/resolve/main/added_tokens.json from cache at /Users/d.tsimerman/.cache/huggingface/transformers/8969e3b2f8db62ca0eadb45ff1085c9cc2989ac3f7db0e14af9977ba83a0573d.c0d3a65a693915a101abad0f4d9e99b33fc983fd2037e92cfb6788725c50a700
loading file https://huggingface.

In [22]:
inputs = tokenizer('@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как у тебядела? @@ВТОРОЙ@@ ', return_tensors='pt')
generated_token_ids = model.generate(**inputs)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/d.tsimerman/blowjob/pycon-chit-chat/.venv/lib/python3.8/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как у тебядела? @@ВТОРОЙ@@      ']